In [1]:
import pandas as pd
import openai
import time
from tqdm import tqdm

tqdm.pandas()

In [2]:
df = pd.read_csv("data/preprocessed_commercial_area.csv")
cluster_df = pd.read_csv("data/cluster_data.csv")

In [3]:
OPENAI_API_KEY = ""
openai.api_key = OPENAI_API_KEY

In [4]:
# 일부 행정동에 한해서만 진행
# area_lst = {"압구정동", "역삼1동", "역삼2동", "청담동", "수서동", "대치2동"}

# df = df[df["행정구역"].apply(lambda area: True if area in area_lst else False)]

prompt_df = df[["시군구", "행정구역"]].copy()

In [10]:
persona_dict = {
    "음식점": ["한식", "중식", "일식", "양식"],
    "일반학원": ["일반교습학원", "외국어학원", "독서실"],
    "병원": ["일반의원", "치과의원", "한의원", "동물병원"],
    "오락서비스": ["당구장", "골프연습장", "볼링장", "PC방", "전자게임장", "기타오락장", "노래방", "DVD방"],
    "카페": ["제과점", "커피-음료"], # 2개
    "마트": ["슈퍼마켓", "편의점"], # 2개?
    "음식소매": ["주류도매", "미곡판매", "육류판매", "수산물판매", "청과상", "반찬가게"],
    "의류소매": ["일반의류", "한복점", "유아의류", "신발", "가방", "안경", "시계및귀금속"]
}

txt = """

[요청] : 
입력한 내용으로 상권 특성을 반영하여 설명을 해줘.

[설명 방법] :
- 백화점 마케터의 관점에서 볼 때 2단어 이상으로 구성된 상권 이름을 작명해줘. 이때 특정 지역 이름이 나오지 않도록 해줘.
- 작명된 이름을 바탕으로 백화점 마케터가 필요로 하는 내용(예를 들면 해당 상권의 특징은 무엇이며 어떤 것이 필요로 할지)을 200자 이내로 상세하게 작성해줘.

[결과 예시]
이름 : ...
내용 : ..."""

In [11]:
example_prompt = """[역할] : 너는 백화점 운영 및 마케팅을 10년 이상 해왔다. 각 상권별로 유동인구, 소비 특성, 소득, 주변 시설 등 모든 데이터를 파악하고 있는 상태이다. 

[내용] :
한식음식점 전체 점포수는 402.2개로 서울 행정동내에서 매우 많은편이며, 작년 대비 감소하는 추세이다.

중식음식점 전체 점포수는 45.6개로 서울 행정동내에서 매우 많은편이며, 작년 대비 증가하는 추세이다.

일식음식점 전체 점포수는 74.2개로 서울 행정동내에서 매우 많은편이며, 작년 대비 증가하는 추세이다.

양식음식점 전체 점포수는 103.7개로 서울 행정동내에서 매우 많은편이며, 작년 대비 증가하는 추세이다.


[요청] : 
입력한 내용으로 상권 특성을 반영하여 설명을 해줘.

[설명 방법] :
- 백화점 마케터의 관점에서 볼 때 2단어 이상으로 구성된 상권 이름을 작명해줘. 이때 특정 지역 이름이 나오지 않도록 해줘.
- 작명된 이름을 바탕으로 백화점 마케터가 필요로 하는 내용(예를 들면 해당 상권의 특징은 무엇이며 어떤 것이 필요로 할지)을 200자 이내로 상세하게 작성해줘.

[결과 예시]
이름 : ...
내용 : ..."""

example_result = """이름: 먹거리 거리

먹거리 거리는 한식, 중식, 일식, 양식 전체 점포수가 매우 많은 상권입니다. 특히, 한식 음식점은 해당 지역에서 가장 많은 비중을 차지하고, 작년 대비 감소하는 추세입니다. 또한, 인근에는 다양한 먹거리를 제공하는 마켓이 위치하고 있어, 소비자들은 다양한 음식을 즐길 수 있습니다. 따라서, 해당 상권에서는 한식 음식점의 경쟁이 치열해지고, 소비자들이 더욱 다양한 메뉴와 경험을 원할 수 있으므로, 음식점들은 새로운 시도와 창의적인 마케팅이 필요할 것입니다. 또한, 먹거리를 즐기는 소비자들이 많기 때문에, 음식점들은 질 좋은 식재료와 맛, 서비스에 더욱 신경을 써야 합니다."""

In [12]:
def get_story(prompt):
    model = "gpt-3.5-turbo"
    messages = [
        {"role": "user", "content": example_prompt},
        {"role": "assistant", "content": example_result},
        {"role": "user", "content": prompt},
    ]
    response = openai.ChatCompletion.create(model=model, 
                                            messages=messages,)
    return response["choices"][0]["message"]["content"]

def get_rank(data):
    if data <= 10:
        return "매우 많은편", "매우 높은편", "매우 긴편"
    elif data <= 30:
        return "많은편", "높은편", "긴편"
    elif data <= 70:
        return "적당히 있는편", "보통인편", "보통인편"
    elif data <= 90:
        return "적은편", "낮은편", "짧은편"
    else:
        return "매우 적은편", "매우 낮은편", "매우 짧은편"

def make_prompt(data):
    cat1, cat2 = data.index[1].split(" ")[-2:]
    
    prompt = f"""
{cat2}수는 {data[f'2022 전체 점포수 {cat1} {cat2}']}개로 서울에서 {get_rank(data[f'2022 전체 점포수 {cat1} {cat2} 등급'])[0]}이며, 작년 대비 {'증가하는' if data[f'전체 점포수 {cat1} {cat2} 변화율'] > 0 else ('비슷한' if data[f'전체 점포수 {cat1} {cat2} 변화율'] == 0 else '감소하는')} 추세이다.
"""
# 신생기업 생존율은 {get_rank(data[[f'2022 신생기업 1년 생존율 {cat1} {cat2}', f'2022 신생기업 3년 생존율 {cat1} {cat2}', f'2022 신생기업 5년 생존율 {cat1} {cat2}']].mean())[1]}, 최근 10년기준 평균 영업기간은 {get_rank(data[f'2022 최근 10년기준 {cat1} {cat2} 등급'])[2]}이다.
# 개업율은 {data[f'2022 개업률 {cat1} {cat2}']}%로 서울 행정동내에서 {get_rank(data[f'2022 개업률 {cat1} {cat2} 등급'])[1]}, 폐업율은 {data[f'2022 폐업률 {cat1} {cat2}']}%로 서울 행정동내에서 {get_rank(data[f'2022 폐업률 {cat1} {cat2} 등급'])[1]}이다.
    
    tmp_prompt[data[category]] += prompt

for category, search_values in persona_dict.items():
    print(category)
    tmp_prompt = {k: "[역할] : 너는 백화점 운영 및 마케팅을 10년 이상 해왔다. 각 상권별로 유동인구, 소비 특성, 소득, 주변 시설 등 모든 데이터를 파악하고 있는 상태이다. \n\n[내용] :" for k in cluster_df[category].unique()}
    for tmp_value in search_values:
        tmp_columns = [col for col in df.columns if tmp_value in col]
        tmp_df = pd.merge(df[["시군구", "행정구역"] + tmp_columns], cluster_df[["시군구", "행정구역", category]], on=["시군구", "행정구역"]).fillna(0)
        tmp_df = round(tmp_df.groupby(category).mean().reset_index(), 1)
        tmp_df.apply(make_prompt, axis=1)
    
    tmp_prompt = {k: v + txt for k, v in tmp_prompt.items()}
    
    for k, v in tmp_prompt.items():
        print("="*100)
        print(v)
        tmp_prompt[k] = get_story(v)
        print("="*100)
        print(tmp_prompt[k])

        time.sleep(20)
        print()
        
    prompt_df[category + " story"] = cluster_df[category].apply(lambda x: tmp_prompt[x])

음식점
[역할] : 너는 백화점 운영 및 마케팅을 10년 이상 해왔다. 각 상권별로 유동인구, 소비 특성, 소득, 주변 시설 등 모든 데이터를 파악하고 있는 상태이다. 

[내용] :
한식음식점수는 59.7개로 서울에서 적은편이며, 작년 대비 감소하는 추세이다.

중식음식점수는 5.4개로 서울에서 적은편이며, 작년 대비 비슷한 추세이다.

일식음식점수는 5.5개로 서울에서 적은편이며, 작년 대비 증가하는 추세이다.

양식음식점수는 5.9개로 서울에서 적은편이며, 작년 대비 증가하는 추세이다.


[요청] : 
입력한 내용으로 상권 특성을 반영하여 설명을 해줘.

[설명 방법] :
- 백화점 마케터의 관점에서 볼 때 2단어 이상으로 구성된 상권 이름을 작명해줘. 이때 특정 지역 이름이 나오지 않도록 해줘.
- 작명된 이름을 바탕으로 백화점 마케터가 필요로 하는 내용(예를 들면 해당 상권의 특징은 무엇이며 어떤 것이 필요로 할지)을 200자 이내로 상세하게 작성해줘.

[결과 예시]
이름 : ...
내용 : ...
이름: 식도락 길

식도락 길은 서울에서 한식 음식점이 가장 적은 상권 중 하나이며, 작년 대비 감소하는 추세입니다. 그러나, 일식, 양식 음식점은 작년 대비 증가하고 있으며, 특히 일식 음식점의 수가 적은 상황에서 증가하는 것은 유망한 신호입니다. 해당 상권은 소비자들이 식도락을 하며 산책할 수 있는 공간으로 인접한 공원과 관련 시설이 많아, 더욱 매력적입니다. 따라서, 식도락 길에서는 한식 음식점들은 경쟁이 조금 더 치열해질 수 있습니다. 소비자들은 다양한 음식을 즐기려 할 것이므로 다양한 메뉴와 새로운 음식점 개발, 미식여행 등 새로운 시도가 필요할 것입니다. 일식, 양식 음식점들은 수가 적지만, 새로운 메뉴와 매력적인 맛과 분위기를 제공하여 소비자들의 관심을 끌어내는 마케팅이 필요할 것입니다.

[역할] : 너는 백화점 운영 및 마케팅을 10년 이상 해왔다. 각 상권별로 유동인구, 소비 특성, 소득, 주변 시설 등 모든 데이터를 파악하고 있는 상

In [13]:
prompt_df.to_csv("data/prompt_story.csv",index=False)